# Basic MCMC examples

In [ ]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats

plt.rcParams['xtick.minor.visible'], plt.rcParams['xtick.top'] = True,True 
plt.rcParams['ytick.minor.visible'], plt.rcParams['ytick.right'] = True,True 
plt.rcParams['xtick.direction'], plt.rcParams['ytick.direction'] = 'in','in' 

In [ ]:
plt.rcParams['font.size'] = 18 

# Functions to create/plot PDF

In [ ]:
def StrangePdf(x,centres,covars,weights) :
    '''Gives a generic strange shaped pdf for the Metropolis example'''
    PDF = np.array([])
    if(x.ndim == 1) : PDF = np.zeros_like(x[0])
    if(x.ndim == 2) : PDF = np.zeros_like(x[:,0])
    if(x.ndim == 3) : PDF = np.zeros_like(x[:,:,0])
    if(x.ndim > 3 ) : return 0
    #PDF = np.zeros_like(x[:,:,0])
    for i in range(len(centres)) :
        PDF += weights[i]*scipy.stats.multivariate_normal.pdf(x,centres[i],covars[i])
    return PDF / np.sum(weights)

In [ ]:
def PlotPdf(pdf,colors=None) :
    '''Plots the pdf given (in a pre-approved range)'''
    x, y = np.mgrid[0:4:.01, 0:7:.01]
    pos = np.empty(x.shape + (2,))
    pos[:, :, 0] = x
    pos[:, :, 1] = y
    plt.contour(x, y, pdf(pos),colors=colors)
    plt.xlabel('x')
    plt.ylabel('y')

In [ ]:
def specificStrangePdf (pos) :
    '''Particular choice of pdf for example'''
    centres = [[1.2,2.5],[2.1,4],[3.0,5]]
    covars = [[[0.25,-0.2],[-0.2,0.5]],
              [[0.15,0.1],[0.1,1.5]],
              [[0.3,0.],[0.,0.2]]]
    weights = [0.4,0.7,0.15]
    return StrangePdf(pos,centres,covars,weights)

# Metropolis-Hastings algorithm

In [ ]:
def SimpleMetropolisHastings(pdf, proposalcovar, initialvalue, initialpdfvalue) :
    '''Takes a step in a Metropolis-Hastings algorithm
    
    Returns new step in chain, pdf value of that step, pdf value of 
    proposal step (these may be the same)'''
    # generate proposal state
    dval = np.dot(proposalcovar, np.random.randn(len(initialvalue)))
    proposalvalue = initialvalue + dval
    # Find pdf value
    newpdfvalue = pdf(proposalvalue)
    # Make step if better
    if newpdfvalue > initialpdfvalue : 
        return proposalvalue,newpdfvalue,newpdfvalue
    else :
        # Allow for possible step if worse
        tester = np.random.rand() # U(0,1)
        if tester < newpdfvalue/initialpdfvalue : 
            return proposalvalue,newpdfvalue,newpdfvalue
        else :
            return initialvalue, initialpdfvalue,newpdfvalue
            

# Show the pdf as a contour plot

In [ ]:

PlotPdf(specificStrangePdf)
plt.show()


# Run for a few steps

In [ ]:
nchain = 200
chain = np.zeros([nchain,2])

chain[0] = [10,2]
# covariance matrix of propsal 
propcovar = [[0.1,0],[0,0.2]]
pdfvalue = specificStrangePdf(chain[0])

# compute mean acceptance ratio
acceptanceRatio = 0.

#Run Metropolis Hastings
for i in range(1,nchain) :
    oldpdfvalue = pdfvalue
    chain[i],pdfvalue,steppdfvalue = SimpleMetropolisHastings(specificStrangePdf,propcovar,chain[i-1],pdfvalue)
    acceptanceRatio += np.min([1,steppdfvalue/oldpdfvalue])

print('mean a=',acceptanceRatio/(nchain-1))

PlotPdf(specificStrangePdf)
plt.plot(chain[:,0],chain[:,1])
plt.show()


# Run longer and from a worse starting point

In [ ]:
nchain = 2000
chain = np.zeros([nchain,2])

chain[0] = [10,2]
# covariance matrix of propsal 
propcovar = [[0.1,0],[0,0.2]]
pdfvalue = specificStrangePdf(chain[0])

# compute mean acceptance ratio
acceptanceRatio = 0.

#Run Metropolis Hastings
for i in range(1,nchain) :
    oldpdfvalue = pdfvalue
    chain[i],pdfvalue,steppdfvalue = SimpleMetropolisHastings(specificStrangePdf,propcovar,chain[i-1],pdfvalue)
    acceptanceRatio += np.min([1,steppdfvalue/oldpdfvalue])

print('mean a=',acceptanceRatio/(nchain-1))

PlotPdf(specificStrangePdf)
plt.plot(chain[:,0],chain[:,1])
plt.show()


# Check for convergence by-eye: show the trails

Note initial burn-in period, and strong correlation on short time-scales

In [ ]:
plt.plot(chain)
plt.xlabel('step')
plt.ylabel('x,y')
plt.show()

# Run longer

In [ ]:
nchain = 20000
chain = np.zeros([nchain,2])

chain[0] = [10,2]
# covariance matrix of propsal 
propcovar = [[0.1,0],[0,0.2]]
pdfvalue = specificStrangePdf(chain[0])

# compute mean acceptance ratio
acceptanceRatio = 0.

#Run Metropolis Hastings
for i in range(1,nchain) :
    oldpdfvalue = pdfvalue
    chain[i],pdfvalue,steppdfvalue = SimpleMetropolisHastings(specificStrangePdf,propcovar,chain[i-1],pdfvalue)
    acceptanceRatio += np.min([1,steppdfvalue/oldpdfvalue])

print('mean a=',acceptanceRatio/(nchain-1))

PlotPdf(specificStrangePdf)
plt.plot(chain[:,0],chain[:,1])
plt.show()


# Check for convergence by-eye: show the trails

Note strong correlation on short time-scales

In [ ]:
plt.plot(chain)
plt.xlabel('step')
plt.ylabel('x,y')
plt.show()

# Run with a better tuned covariance matrix

In [ ]:
nchain = 200
chain = np.zeros([nchain,2])

chain[0] = [3.5,2]
# covariance matrix of propsal 
propcovar = [[1.5,0],[0,2.]]
pdfvalue = specificStrangePdf(chain[0])

# compute mean acceptance ratio
acceptanceRatio = 0.

for i in range(1,nchain) :
    oldpdfvalue = pdfvalue
    chain[i],pdfvalue,steppdfvalue = SimpleMetropolisHastings(specificStrangePdf,propcovar,chain[i-1],pdfvalue)
    acceptanceRatio += np.min([1,steppdfvalue/oldpdfvalue])

print('mean a=',acceptanceRatio/(nchain-1))

PlotPdf(specificStrangePdf)
plt.plot(chain[:,0],chain[:,1], '.')
plt.show()


# Run longer

In [ ]:
nchain = 2000
chain = np.zeros([nchain,2])

chain[0] = [3.5,2]
# covariance matrix of propsal 
propcovar = [[1.5,0],[0,2.]]
pdfvalue = specificStrangePdf(chain[0])

# compute mean acceptance ratio
acceptanceRatio = 0.

for i in range(1,nchain) :
    oldpdfvalue = pdfvalue
    chain[i],pdfvalue,steppdfvalue = SimpleMetropolisHastings(specificStrangePdf,propcovar,chain[i-1],pdfvalue)
    acceptanceRatio += np.min([1,steppdfvalue/oldpdfvalue])

print('mean a=',acceptanceRatio/(nchain-1))

PlotPdf(specificStrangePdf)
plt.plot(chain[:,0],chain[:,1], '.')
plt.show()


# Run longer

In [ ]:
nchain = 20000
chain = np.zeros([nchain,2])

chain[0] = [3.5,2]
# covariance matrix of propsal 
propcovar = [[1.5,0],[0,2.]]
pdfvalue = specificStrangePdf(chain[0])

# compute mean acceptance ratio
acceptanceRatio = 0.

for i in range(1,nchain) :
    oldpdfvalue = pdfvalue
    chain[i],pdfvalue,steppdfvalue = SimpleMetropolisHastings(specificStrangePdf,propcovar,chain[i-1],pdfvalue)
    acceptanceRatio += np.min([1,steppdfvalue/oldpdfvalue])

print('mean a=',acceptanceRatio/(nchain-1))

PlotPdf(specificStrangePdf)
plt.plot(chain[:,0],chain[:,1], '.')
plt.show()


# trail plot again

Looks much nicer now

In [ ]:
plt.plot(chain)
plt.xlabel('step')
plt.ylabel('x,y')
plt.show()

# Find for a large number of points 

## Discard some of the early points in the chain (burn-in)
### This ensures that the results are ~independent of the starting point

In [ ]:
# Number of points in my MCMC chain
nchain = 50000
chain = np.zeros([nchain,2])
# start point
chain[0] = [3,2]
# covariance matrix of propsal 
propcovar = [[1.5,0],[0,2.]]
pdfvalue = specificStrangePdf(chain[0])


# compute mean acceptance ratio
acceptanceRatio = 0.

for i in range(1,nchain) :
    oldpdfvalue = pdfvalue
    chain[i],pdfvalue,steppdfvalue = SimpleMetropolisHastings(specificStrangePdf,propcovar,
                                                              chain[i-1],pdfvalue)
    acceptanceRatio += np.min([1,steppdfvalue/oldpdfvalue])

# here we ignore the first 20% of values in the chain
print('mean values: ', np.mean(chain[nchain//5:,0]), np.mean(chain[nchain//5:,1]))
    
#take only last 80%
plt.hist2d(chain[nchain//5:,0],chain[nchain//5:,1],40,density=True,cmap='plasma')
cbar = plt.colorbar()
cbar.set_label('f(x,y)')
PlotPdf(specificStrangePdf,'k')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

In [ ]:
 
#take only last 80%
plt.hist2d(chain[nchain//5:,0],chain[nchain//5:,1],30,density=True,cmap='plasma')
cbar = plt.colorbar()
cbar.set_label('f(x,y)')
#PlotPdf(specificStrangePdf,'k')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

# And then it's trivial to get histograms of $f_X(x)$ and $f_Y(y)$

In [ ]:
plt.gcf().set_size_inches(10,4)
plt.subplot(121)
plt.hist(chain[nchain//5:,0],50,density=True)
plt.xlabel('x')
plt.ylabel('f(x)')
plt.subplot(122)
plt.hist(chain[nchain//5:,1],50,density=True)
plt.xlabel('y')
plt.ylabel('f(y)')
plt.tight_layout()
plt.show()

# Or averages and uncertainties


In [ ]:
print('E(x): %.2f' % np.mean(chain[nchain//5:,0]))
print('E(y): %.2f' % np.mean(chain[nchain//5:,1]))


In [ ]:
print('Covariance matrix:\n', np.cov(chain[nchain//5:,:].T))

# We can make a corner plot with the corner package

This shows us the 2D marginal distributions of each pair of parameters as 2D histograms, plus the 1D marginal distributions as regular histograms. On the latter we also mark 16%, 50% and 84% quantiles

In [ ]:
import corner

In [ ]:
corner.corner(chain,labels=['x','y'],quantiles=[0.16,0.5,0.84])
plt.show()